In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pathlib import Path
import os
import shutil

In [7]:
input_path = r'C:\Users\Wojtek\Documents\Projects\dog-expert\datasets\train'
csv_path = r'C:\Users\Wojtek\Documents\Projects\dog-expert\datasets\labels.csv'
output_path = r'C:\Users\Wojtek\Documents\Projects\dog-expert\datasets\stanford'

In [8]:
def get_stem(filename):
    stem = Path(filename).stem
    return stem

In [9]:
def organize_classes(input_path, output_path, file_dict):
    breeds_found = set()
    for file_stem, breed in file_dict.items():
        if breed not in breeds_found:
            if not os.path.exists(os.path.join(output_path, breed)):
                os.makedirs(os.path.join(output_path, breed))
            breeds_found.add(breed)
        shutil.copy( os.path.join( input_path, ''.join( [file_stem, '.jpg'] ) ), os.path.join( output_path, breed, ''.join([file_stem, '.jpg']) ))
    return None


In [3]:
def generate_class_mapping(dataset_path):
    class_mapping = dict()
    count = 0
    for dirr in os.listdir(dataset_path):
        class_mapping[dirr] = count
        count += 1
    return class_mapping
    

In [4]:
class_mapping = generate_class_mapping(r'datasets\stanford')

In [7]:
import pickle

In [9]:

file_to_write = open('class_mapping.pkl', "wb")
pickle.dump(class_mapping, file_to_write)

In [10]:
labels = pd.read_csv(csv_path)

In [11]:
file_dict = dict(zip(labels['id'].values, labels['breed'].values))

In [12]:
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [13]:
a = organize_classes(input_path, output_path, file_dict)

In [9]:
train_data

NameError: name 'train_data' is not defined

In [1]:
class_mapping = dict()

In [3]:
import os

In [4]:
count = 0
for dirr in os.listdir(r'datasets\stanford2'):
    class_mapping[dirr] = count
    count +=1

In [5]:
class_mapping

{'affenpinscher': 0,
 'afghan_hound': 1,
 'african_hunting_dog': 2,
 'airedale': 3,
 'american_staffordshire_terrier': 4,
 'appenzeller': 5,
 'australian_terrier': 6,
 'basenji': 7,
 'basset': 8}

In [8]:
p = r'datasets\stanford2'

In [17]:
count = 0
for dirr in os.listdir(p):
    full_p = os.path.join(p, dirr)
    for file in os.listdir(full_p):
        file_path = os.path.join(full_p, file)
        contents = tf.io.read_file(file_path)
        img = tf.io.decode_image(contents, channels=3)
        count+=1

In [18]:
count

835

In [14]:
file_path

'datasets\\stanford2\\affenpinscher\\00ca18751837cd6a22813f8e221f7819.jpg'

In [5]:
import dog_expert
from dog_expert import DataLoader, Preprocessor
import tensorflow as tf


from tensorflow.keras import layers
from tensorflow.keras import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import Xception

IMAGE_SIZE = (256, 256)

In [ ]:
preprocessor = Preprocessor()
preprocessor.cast(dtype=tf.float32).normalize()

class_mapping = {'affenpinscher': 0,
                 'afghan_hound': 1,
                 'african_hunting_dog': 2,
                 'airedale': 3,
                 'american_staffordshire_terrier': 4,
                 'appenzeller': 5,
                 'australian_terrier': 6,
                 'basenji': 7,
                 'basset': 8}

data_loader = DataLoader(dataset_dir=r'datasets\stanford2',
                         batch_size=8, class_mapping=class_mapping, preprocessor=preprocessor)

train = data_loader.train_dataset
val = data_loader.val_dataset

print(len(train))
print(len(val))

base_model = Xception(weights=None, include_top=False, input_shape=(*IMAGE_SIZE, 3))
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
# and a logistic layer -- let's say we have 200 classes
predictions = layers.Dense(9, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(loss='categorical_crossentropy', optimizer=Adam(
    learning_rate=0.03), metrics=['accuracy'])



In [10]:


for x, y in train.as_numpy_iterator():
    a = x
    b = y

In [13]:
b['target'].shape

(3, 9)

In [14]:
a['data_point'].shape

(3, 256, 256, 3)

In [7]:
history = model.fit(train, validation_data=val,
                    epochs=50)

Epoch 1/50


ValueError: Found unexpected keys that do not correspond to any Model output: dict_keys(['target']). Expected: ['dense_1']